# Init Milvus connection

In case of using custom configurations, create a `.env` file in the root directory.

In [1]:
import os

from pymilvus import connections, db
from load_dotenv import load_dotenv

load_dotenv()

MILVUS_HOST = os.getenv("MILVUS_HOST") or "localhost"
MILVUS_PORT = os.getenv("MILVUS_PORT") or 19530
MILVUS_DB = os.getenv("MILVUS_DB") or "default"

# Connect to Milvus
conn = connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)


# Setup database
# db.create_database(MILVUS_DB) # run only once
db.using_database(MILVUS_DB)

db.set_properties(db_name=MILVUS_DB, properties={"database.max.collections": 100})


db.describe_database(MILVUS_DB)


{'name': 'default', 'database.max.collections': '100'}

# Manage collections

In [2]:
COLLECTION_NAME = "text_collection_lab"
EMBEDDING_LENGTH = 1024


The schema used to create a new collection is defined by `CollectionSchema`.

You need to [define FieldSchema](https://milvus.io/docs/schema.md) before defining CollectionSchema.

In [ ]:
from pymilvus import Collection, DataType, CollectionSchema, FieldSchema

# Define field schema
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True, description="primary key")
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=EMBEDDING_LENGTH, description="embedding vector")
text_field = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535, description="text field") # require max_length defined


# Define collection schema
schema = CollectionSchema(fields=[id_field, embedding_field, text_field], description="example collection schema")

# Create collection
collection = Collection(name=COLLECTION_NAME, schema=schema)


Milvus requires defining the index column before loading. Refer to [index types and corresponding parameters](https://milvus.io/docs/index-vector-fields.md).

In [4]:
# Define collection index
from pymilvus import IndexType

index_params = {"nlist": 16384,
                'index_type': IndexType.IVF_SQ8,
                'metric_type': 'IP',
                'params': {'nlist': 16384}
                }

collection.create_index(field_name="embedding", index_params=index_params)

Status(code=0, message=)

Loading a collection is the prerequisite to conducting similarity searches and queries in collections. 

In [5]:
collection.load()

Searches and queries are memory-intensive operations. To save the cost, you are advised to release the collections that are currently not in use.

In [30]:

collection.release()

In [31]:
# Drop collection
# collection.drop()